<a href="https://colab.research.google.com/github/Andrian0s/ML4NLP1-2024-Tutorial-Notebooks/blob/main/exercises/ex1/ex1_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML4NLP1

## Starting Point for Exercise 1, part II



This notebook is supposed to serve as a starting point and/or inspiration when starting exercise 1, part II.



One of the goals of this exercise is o make you acquainted with **skorch**. You will probably need to consult the [documentation](https://skorch.readthedocs.io/en/stable/).

# Installing skorch and loading libraries

In [1]:
import subprocess



# Installation on Google Colab

try:

    import google.colab

    subprocess.run(['python', '-m', 'pip', 'install', 'skorch'])

except ImportError:

    pass

In [2]:
!pip install skorch

!pip install gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 5.3 MB/s eta 0:00:0000:01


In [3]:
import torch

from torch import nn

import torch.nn.functional as F

from skorch import NeuralNetClassifier



import pandas as pd

import numpy as np

import csv

import re

import string

from collections import defaultdict



# Set seed for reproducibility

seed = 42

np.random.seed(seed)

torch.manual_seed(seed)

torch.cuda.manual_seed(seed)

## Training a classifier and making predictions

In [4]:
# Download dataset

!gdown 1QP6YuwdKFNUPpvhOaAcvv2Pcp4JMbIRs # x_train

!gdown 1QVo7PZAdiZKzifK8kwhEr_umosiDCUx6 # x_test

!gdown 1QbBeKcmG2ZyAEFB3AKGTgSWQ1YEMn2jl # y_train

!gdown 1QaZj6bI7_78ymnN8IpSk4gVvg-C9fA6X # y_test

Downloading...
From: https://drive.google.com/uc?id=1QP6YuwdKFNUPpvhOaAcvv2Pcp4JMbIRs
To: /kaggle/working/x_train.txt
100%|██████████████████████████████████████| 64.1M/64.1M [00:01<00:00, 40.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QVo7PZAdiZKzifK8kwhEr_umosiDCUx6
To: /kaggle/working/x_test.txt
100%|██████████████████████████████████████| 65.2M/65.2M [00:00<00:00, 70.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QbBeKcmG2ZyAEFB3AKGTgSWQ1YEMn2jl
To: /kaggle/working/y_train.txt
100%|█████████████████████████████████████████| 480k/480k [00:00<00:00, 108MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QaZj6bI7_78ymnN8IpSk4gVvg-C9fA6X
To: /kaggle/working/y_test.txt
100%|████████████████████████████████████████| 480k/480k [00:00<00:00, 98.2MB/s]


In [5]:
with open(f'x_train.txt') as f:

    x_train = f.read().splitlines()

with open(f'y_train.txt') as f:

    y_train = f.read().splitlines()

with open(f'x_test.txt') as f:

    x_test = f.read().splitlines()

with open(f'y_test.txt') as f:

    y_test = f.read().splitlines()

In [6]:
# Combine x_train and y_train into one dataframe

train_df = pd.DataFrame({'text': x_train, 'label': y_train})

# Write train_df to csv with tab as separator

train_df.to_csv('train_df.csv', index=False, sep='\t')

# Comibne x_test and y_test into one dataframe

test_df = pd.DataFrame({'text': x_test, 'label': y_test})

# Inspect the first 5 items in the train split

train_df.head()

,text,label
0,Klement Gottwaldi surnukeha palsameeriti ning ...,est
1,"Sebes, Joseph; Pereira Thomas (1961) (på eng)....",swe
2,भारतीय स्वातन्त्र्य आन्दोलन राष्ट्रीय एवम क्षे...,mai
3,"Après lo cort periòde d'establiment a Basilèa,...",oci
4,ถนนเจริญกรุง (อักษรโรมัน: Thanon Charoen Krung...,tha


### Data preparation



Prepare your dataset for this experiment using the same method as you did in part 1.



Get a subset of the train/test data that includes 20 languages. Include English, German, Dutch, Danish, Swedish, Norwegian, and Japanese, plus 13 additional languages of your choice based on the items in the list of labels.



Don't forget to encode your labels using the adjusted code snippet from part 1!


In [7]:
# TODO: Create your train/test subsets of languages

# Note, make sure these are the same as what you used in Part 1!



from sklearn.model_selection import train_test_split



# TODO: Create your train/test subsets of languages

language_filter = ['eng','deu','nld','dan','swe','nob','jpn', #basics

                   'fra', 'spa', 'rus', 'por', 'ita', 'kor', 'ara', 'zho', 'hin', 'tam', 'tha', 'vie', 'fin' #additionals

                   ]

# Filter x and y based on the language filter

filtered_x = [text for text,label in zip(x_train + x_test,y_train + y_test) if label in language_filter]

filtered_y = [label for label in y_train + y_test if label in language_filter]



# Split the train/test data into 8:2

x_train,x_test,y_train,y_test = train_test_split(filtered_x,filtered_y,test_size = 0.2,random_state=42)



#display

print(x_train[:5])

print(y_train[:5])

['銀行券は帝国国庫及びドイツ帝国銀行(Reichsbank)から発行され、帝国のいくつかの構成国の銀行からも発行された。帝国国庫発行の帝国紙幣(Reichskassenschein)は5、10、20、50マルクが発行された一方、ドイツ帝国銀行券(Reichsbanknote)は20、50、100、1000マルクが発行された。1914年以降に発行されたこれらの銀行券はパピエルマルクと呼ばれる。', 'في عام 2007، كرئيس أساقفة و كاردينال بوينس آيرس، قدم بيرجوليو النسخة النهائية من البيان المشترك الصادر عن أساقفة أمريكا اللاتينية المسمى "وثيقة أباريسيدا" بعد إقراره من قبل البابا بندكت السادس عشر. نصت الوثيقة على ضرورة الامتثال و قبول تعاليم الكنيسة ضد "جرائم نكراء" مثل الإجهاض والقتل الرحيم: "نأمل أن المشرعين ورؤساء الحكومات، والعاملين في مجال الصحة، سيدركون كرامة الحياة الإنسانية وأهمية العائلة في شعوبنا، و سيدافعون عن حمايتها من جرائم نكراء مثل الإجهاض والقتل الرحيم، وهذه هي مسؤوليتهم. ونحن نلزم أنفسنا "تماسك إفخارستي"، بما معناه، يجب أن نكون واعين بأن الناس لا يستطيعون الحصول على القربان المقدس وفي الوقت نفسه هم يعملون ضد الوصايا، ولا سيما عندما يوافقون على الإجهاض والقتل الرحيم، وغيرها من الجرائم الخطيرة ضد الحياة والعائلة، وهو ينطبق بشكل خاص على مسؤولية المشرعين والحكام، والعاملين

In [8]:
# TODO: Use your adjusted code from part 1 to encode the labels again

from sklearn.preprocessing import LabelEncoder



label_encoder = LabelEncoder().fit(y_train)

y_train, y_test = label_encoder.transform(y_train), label_encoder.transform(y_test)

print(label_encoder.classes_)

print(y_train)

print(y_test)

['ara' 'dan' 'deu' 'eng' 'fin' 'fra' 'hin' 'ita' 'jpn' 'kor' 'nld' 'nob'
 'por' 'rus' 'spa' 'swe' 'tam' 'tha' 'vie' 'zho']
[ 8  0  4 ... 19 18  1]
[ 7 15 12 ...  1  1  0]


### Feature Extraction

In [9]:
# First, we extract some simple features as input for the neural network

from sklearn.feature_extraction.text import CountVectorizer



vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 2), max_features=100, binary=True)

X = vectorizer.fit_transform(x_train)

In [10]:
# We need to change the datatype to make it play nice with pytorch

X = X.astype(np.float32)

y = y_train.astype(np.int64)

In the following, we define a vanilla neural network with two hidden layers. The output layer should have as many outputs as there are classes. In addition, it should have a nonlinearity function.

In [11]:
# TODO: In the following, you can find a small (almost) working example of a neural network.

# Unfortunately, again, the cat messed up some of the code. Please fix the code such that it is executable. (Hint: the input and output sizes look a bit weird...)



class ClassifierModule(nn.Module):

    def __init__(

        self,

        num_units=200,

        nonlin=F.relu,

        num_classes=20,

        input_size=100,

    ):

        super(ClassifierModule, self).__init__()

        self.num_units = num_units

        self.nonlin = nonlin



        self.dense0 = nn.Linear(input_size, num_units)

        self.nonlin = nonlin

        self.dense1 = nn.Linear(num_units, 50)

        self.output = nn.Linear(50, num_classes)



    def forward(self, X, **kwargs):

        X = self.nonlin(self.dense0(X))

        X = F.relu(self.dense1(X))

        X = self.output(X)

        return X.squeeze(dim=1)


In [12]:
# Initalise the neural net classifier.

net = NeuralNetClassifier(

    ClassifierModule(

        input_size=X.shape[1],

        num_units=200,

        num_classes=len(label_encoder.classes_),

        nonlin=F.relu,

    ),

    max_epochs=20,

    criterion=nn.CrossEntropyLoss(),

    lr=0.1,

    device='cuda',  # comment this to train with CPU

)

In [13]:
# Train the classifier

net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.7376       0.2303        2.4827  1.9744
      2        2.1024       0.4122        1.7188  1.7499
      3        1.5182       0.5319        1.3270  1.7271
      4        1.2503       0.6306        1.1424  1.7992
      5        1.0864       0.6506        1.0249  1.6949
      6        0.9856       0.6606        0.9579  1.6832
      7        0.9264       0.6700        0.9192  1.7867
      8        0.8891       0.6753        0.8941  1.7717
      9        0.8632       0.6787        0.8764  1.7162
     10        0.8434       0.6822        0.8627  1.7720
     11        0.8275       0.6837        0.8523  1.7142
     12        0.8141       0.6875        0.8438  1.7065
     13        0.8027       0.6897        0.8368  1.6811
     14        0.7928       0.6925        0.8307  1.7162
     15        0.7841       0.6937        0.8263  1.7428
     16        0.7762       0.6

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=100, out_features=200, bias=True)
    (dense1): Linear(in_features=200, out_features=50, bias=True)
    (output): Linear(in_features=50, out_features=20, bias=True)
  ),
)

In [14]:
X_test = vectorizer.transform(x_test)

X_test = X_test.astype(np.float32)

y_test_np = np.array(y_test, dtype=np.int64)



y_pred = net.predict(X_test)

test_accuracy = np.mean(y_pred == y_test_np)

print(f"Test Accuracy: {test_accuracy}")


Test Accuracy: 0.682


### Experimenting with a better count vectorizer

In [15]:
vectorizer_updated = CountVectorizer(analyzer='char', ngram_range=(2, 2), max_features=5000, binary=True)

X_cv_updated = vectorizer_updated.fit_transform(x_train)

X_cv_updated = X_cv_updated.astype(np.float32)

y_cv_updted = y_train.astype(np.int64)



net_cv_updated = NeuralNetClassifier(

    ClassifierModule(

        input_size=X_cv_updated.shape[1],

        num_units=200,

        num_classes=len(label_encoder.classes_),

        nonlin=F.relu,

    ),

    max_epochs=20,

    criterion=nn.CrossEntropyLoss(),

    lr=0.1,

    device='cuda',  # comment this to train with CPU

)

In [16]:
net_cv_updated.fit(X_cv_updated, y_cv_updted)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.3122       0.6966        1.2376  1.9263
      2        0.7029       0.8925        0.3870  1.8889
      3        0.2740       0.9647        0.2131  1.9559
      4        0.1582       0.9731        0.1500  1.9326
      5        0.1123       0.9766        0.1245  1.9341
      6        0.0886       0.9781        0.1110  1.9176
      7        0.0732       0.9788        0.1026  1.9376
      8        0.0620       0.9794        0.0969  1.9674
      9        0.0534       0.9803        0.0927  1.9149
     10        0.0464       0.9809        0.0898  1.9440
     11        0.0406       0.9816        0.0876  1.9457
     12        0.0358       0.9816        0.0861  1.9044
     13        0.0316       0.9812        0.0850  1.9995
     14        0.0281       0.9816        0.0842  1.9498
     15        0.0250       0.9816        0.0836  1.8980
     16        0.0223       0.9

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=5000, out_features=200, bias=True)
    (dense1): Linear(in_features=200, out_features=50, bias=True)
    (output): Linear(in_features=50, out_features=20, bias=True)
  ),
)

In [17]:
X_cv_updated_test = vectorizer_updated.transform(x_test)

X_cv_updated_test = X_cv_updated_test.astype(np.float32)

y_test_np = np.array(y_test, dtype=np.int64)



y_cv_updated_pred = net_cv_updated.predict(X_cv_updated_test)

test_cv_updated_accuracy = np.mean(y_cv_updated_pred == y_test_np)

print(f"Test Accuracy: {test_cv_updated_accuracy}")

Test Accuracy: 0.98075


### Experimenting with TF-IDF vectorizer instead of count vectorizer

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer



# Use TF-IDF vectorizer for better feature representation

vectorizer_tfidf = TfidfVectorizer(analyzer='char', ngram_range=(2,2), max_features=5000, use_idf=True) # Increased ngram range and max_features

X_tfidf = vectorizer_tfidf.fit_transform(x_train)

X_tfidf = X_tfidf.astype(np.float32)

y_tfidf = y_train.astype(np.int64)





# Initalise the neural net classifier.

net_tfid = NeuralNetClassifier(

    ClassifierModule(

        input_size=X_tfidf.shape[1],

        num_units=200,

        num_classes=len(label_encoder.classes_),

        nonlin=F.relu,

    ),

    max_epochs=20,

    criterion=nn.CrossEntropyLoss(),

    lr=0.1,

    device='cuda',  # comment this to train with CPU

)



# Train the classifier

net_tfid.fit(X_tfidf, y_tfidf)


  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.9956       0.0509        2.9912  1.9132
      2        2.9870       0.1506        2.9810  1.8943
      3        2.9729       0.2853        2.9608  1.8978
      4        2.9367       0.4203        2.8967  1.8823
      5        2.7900       0.2347        2.6222  1.9340
      6        2.4308       0.3503        2.2732  1.9807
      7        2.1525       0.6381        2.0182  1.9009
      8        1.8529       0.7141        1.6706  1.8926
      9        1.4840       0.7234        1.3078  1.8834
     10        1.1609       0.7422        1.0385  1.9115
     11        0.9309       0.7953        0.8462  1.8606
     12        0.7571       0.8653        0.6860  1.8747
     13        0.6040       0.9156        0.5400  1.8755
     14        0.4709       0.9297        0.4230  1.9304
     15        0.3704       0.9400        0.3425  1.8627
     16        0.3023       0.9

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=5000, out_features=200, bias=True)
    (dense1): Linear(in_features=200, out_features=50, bias=True)
    (output): Linear(in_features=50, out_features=20, bias=True)
  ),
)

In [19]:
X_test_tfidf = vectorizer_tfidf.transform(x_test)

X_test_tfidf = X_test_tfidf.astype(np.float32)

y_test_np = np.array(y_test, dtype=np.int64)



y_pred_tfidf = net_tfid.predict(X_test_tfidf)

test_accuracy_tfidf = np.mean(y_pred_tfidf == y_test_np)

print(f"Test Accuracy with TF-IDF: {test_accuracy_tfidf}")


Test Accuracy with TF-IDF: 0.95775


Note, you can also use `GridSearchCV` with `skorch`, but be aware that training a neural network takes much more time.



Play around with 5 different sets of hyperparameters. For example, consider some of the following:



- layer sizes

- activation functions

- regularizers

- early stopping

- vectorizer parameters



Report your best hyperparameter combination. \\

📝❓ What is the effect of your modifcations on validation performance? Discuss potential reasons.

☝ Note, during model development, if you run into the infamous CUDA out-of-memory (OOM) error, try clearing the GPU memory either with `torch.cuda.empty_cache()` or restarting the runtime.

## Grid Search for best hyper parameters with Count Vectorizer

In [20]:
from sklearn.model_selection import GridSearchCV

from skorch.callbacks import EarlyStopping



# Define the parameter grid for GridSearchCV

param_grid = {

    'module__num_units': [100, 200, 300],

    'module__nonlin': [F.relu],

    'module__input_size': [X_cv_updated.shape[1]],

    'lr': [0.01, 0.1],

    'max_epochs': [20, 30],

    'callbacks': [[('EarlyStopping', EarlyStopping(patience=patience))] for patience in [5]]

}



net_cv_gs = NeuralNetClassifier(

    ClassifierModule(

        input_size=X_cv_updated.shape[1],

        num_units=200,

        num_classes=len(label_encoder.classes_),

        nonlin=F.relu,

    ),

    max_epochs=20,

    criterion=nn.CrossEntropyLoss(),

    lr=0.1,

    device='cuda',  # comment this to train with CPU

)

In [21]:
# Create GridSearchCV object

gs = GridSearchCV(net_cv_gs, param_grid, refit=True, cv=3, scoring='accuracy')



# Fit the GridSearchCV object

gs.fit(X_cv_updated, y_cv_updted)



# Print the best parameters and score

print("Best parameters:", gs.best_params_)

print("Best score:", gs.best_score_)



# You can now use the best estimator to make predictions

best_model = gs.best_estimator_

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.9801       0.1012        2.9618  1.2616
      2        2.9412       0.1598        2.9182  1.3189
      3        2.8886       0.2835        2.8548  1.2777
      4        2.8119       0.4025        2.7636  1.2515
      5        2.7086       0.5492        2.6499  1.2733
      6        2.5905       0.7216        2.5287  1.2417
      7        2.4651       0.7610        2.3966  1.2796
      8        2.3203       0.7784        2.2389  1.2790
      9        2.1493       0.7929        2.0583  1.2683
     10        1.9621       0.7990        1.8685  1.2782
     11        1.7709       0.8140        1.6792  1.2573
     12        1.5869       0.8318        1.5034  1.2456
     13        1.4215       0.8477        1.3489  1.2542
     14        1.2771       0.8590        1.2139  1.2554
     15        1.1504       0.8716        1.0954  1.2830
     16        1.0390       0.8

In [22]:
# Predict and get accuracy using best model



y_pred_best = best_model.predict(X_cv_updated_test)

test_accuracy_best = np.mean(y_pred_best == y_test_np)

print(f"Test Accuracy with Best Model: {test_accuracy_best}")

Test Accuracy with Best Model: 0.98


### Best Parameters: 

* Learning Rate: 0.1

* Max Epochs: 30

* Module Input Size: 5000

* Activation: ReLU

* Number of Units: 200

* Early Stopping: 5 Epochs



### The effect of hyperparametrs is significant on the training. Some of the observations are as follows:

* Changing learning rate from 0.01 to 0.1 results in a massive improvement in the validation accuracy. This is seen when the validation accuracy improves from ~30% to nearly 97.1%. This may be because increasing the learning rate resulted in "escaping" the local minima and converge faster.

* Having more units in a layer does not necessarily mean a better accuracy. When the number of units increased from 200 to 300, the accuracy dropped instead of increasing. However, it increased when the number was changed from 100 to 200. This probably implies that 200 units provide sufficient complexity for the model to perform at its best and 300 just leads to overfitting. 

* ReLU outperforms the other activation functions like tanh (not shown on grid search but tested independently). This may be because ReLU does not suffer from the vanishing gradient problem. 

* Increasing max_features for the vectorizer (from 100 to 5000) leads to noticeable increase in accuracy from ~75% to ~97%. More features allowed our models to pick up on even more patterns in the text to make accurate predictions. 

* Early stopping did kick in during training when accuracy stalled for 5 epochs. This allowed the grid search to run at an accelerated pace and find the hyperparameters faster. 






---



📝❓ Write your lab report here addressing all questions in the notebook

# Lab Report



## Introduction



In this lab, we explored the use of neural networks for language classification using the `skorch` library. We experimented with different vectorizers and hyperparameters to improve the model's performance. The dataset consisted of text data in various languages, and the goal was to classify the text into one of the 20 languages.



## Data Preparation



We started by preparing the dataset, which involved:

- Downloading the dataset.

- Combining the training and testing data into dataframes.

- Filtering the data to include only the 20 selected languages.

- Splitting the data into training and testing sets.

- Reorganising the training and test datasets to 80:20 split.

- Encoding the labels using `LabelEncoder`.



## Feature Extraction



We experimented with different feature extraction techniques:

- **Count Vectorizer**: Extracted character-level bigrams with a maximum of 100 and 5000 features.

- **TF-IDF Vectorizer**: Extracted character-level bigrams with a maximum of 5000 features.



## Neural Network Architecture



We did not try to improve the vanilla neural network provided in the code template (other than altering the number of units in the hidden layer). This showed how a simple MLP is capable of outperforming ML techniques introduced in part 1 of the assignment. 



## Experiments and Results



### Initial Experiments



1. **Count Vectorizer with 100 Features**:

    - Achieved a test accuracy of 68.2%.



2. **Count Vectorizer with 5000 Features**:

    - Improved test accuracy to ~98.1%.



3. **TF-IDF Vectorizer**:

    - Achieved a test accuracy of ~95.78%.



### Why choose `Count Vectorizer` over `TF-IDF Vectoizer` for our grid search?

- For our language classification task, the Count Vectorizer showed a slightly higher accuracy (98%) compared to TF-IDF (95.78%). This aligns with expectations for language identification, where the mere presence and frequency of specific character patterns or words are often more indicative of the language than their relative importance across documents.


### Hyperparameter Tuning



We used `GridSearchCV` to find the best hyperparameters. The best parameters were:

- Learning Rate: 0.1

- Max Epochs: 30

- Module Input Size: 5000

- Activation: ReLU

- Number of Units: 200



The best model achieved a test accuracy of ~98%.



### Observations Summary



- **Learning Rate**: Increasing the learning rate from 0.01 to 0.1 resulted in a significant improvement in validation accuracy.

- **Number of Units**: 200 units provided the best performance, while increasing to 300 units led to overfitting.

- **Activation Function**: ReLU was chosen as initial testing done with tanh weren't promising.

- **Vectorizer Features**: Increasing the maximum features for the vectorizer from 100 to 5000 led to a noticeable increase in accuracy.

- **Early Stopping**: Did kick in during grid search where stalled accuracy led to training being stopped and a new combination of hyperparameters being tested.



## Conclusion



The experiments demonstrated the importance of hyperparameter tuning and feature extraction in improving the performance of neural networks for language classification. The best model achieved a test accuracy of ~98%, highlighting the effectiveness of the chosen hyperparameters and vectorizer settings.



---



## Questions



### What is the effect of your modifications on validation performance? Discuss potential reasons.


* Changing learning rate from 0.01 to 0.1 results in a massive improvement in the validation accuracy. This is seen when the validation accuracy improves from ~30% to nearly 97.1%. This may be because increasing the learning rate resulted in "escaping" the local minima and converge faster.

* Having more units in a layer does not necessarily mean a better accuracy. When the number of units increased from 200 to 300, the accuracy dropped instead of increasing. However, it increased when the number was changed from 100 to 200. This probably implies that 200 units provide sufficient complexity for the model to perform at its best and 300 just leads to overfitting. 

* ReLU outperforms the other activation functions like tanh (not shown on grid search but tested independently). This may be because ReLU does not suffer from the vanishing gradient problem. 

* Increasing max_features for the vectorizer (from 100 to 5000) leads to noticeable increase in accuracy from ~75% to ~97%. More features allowed our models to pick up on even more patterns in the text to make accurate predictions. 

* Early stopping did kick in during training when accuracy stalled for 5 epochs. This allowed the grid search to run at an accelerated pace and find the hyperparameters faster. 